In [31]:
import itertools
import json

import jsonschema
import numpy as np
from mitiq import zne
from mitiq.zne.scaling.folding import fold_all, fold_gates_at_random, fold_global
from mitiq.zne.scaling.identity_insertion import insert_id_layers
from mitiq.zne.scaling.layer_scaling import get_layer_folding
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error


# ZNE:


In [32]:
# Load Schema:


def load(schema_path):
    with open(schema_path, "r") as file:
        return json.load(file)


zne_schema = load("zne.json")

In [33]:
# Create a test "batch" experiment:

zne_batch_test = {
    "noise_scaling_factors": [
        [1, 1.25, 1.5],
        [1, 2, 3],
        [2, 4, 6],
    ],  # Noise scaling values
    "noise_scaling_method": ["global"],  # Folding method
    "extrapolation": ["polynomial", "linear"],  # Extrapolation method
}

## Deconstructing a Batch Dictionary:


In [34]:
# Create single experiments from batch object:

# Define a function to make all combinations of experiments from a "batch dictionary" which should be formatted like the test case above
def make_experiment_list(batch_dict):
    # Initialize empty list where we will append each new experiment
    exp_list = []

    # Make list with all combinations of key values from our "batch dictionary"
    combo_list = list(
        itertools.product(
            batch_dict["noise_scaling_factors"],
            batch_dict["noise_scaling_method"],
            batch_dict["extrapolation"],
        )
    )
    # Iterate over the list
    for k in range(len(combo_list)):
        # Initialize single experiment dictionary with keys
        exp = {
            x: set()
            for x in ["noise_scaling_factors", "noise_scaling_method", "extrapolation"]
        }

        # Map each key to its unique value from our list of combinations
        exp["noise_scaling_factors"] = combo_list[k][0]
        exp["noise_scaling_method"] = combo_list[k][1]
        exp["extrapolation"] = combo_list[k][2]

        # Pull out each experiment
        exp_list.append(exp)

    # Returns a list of experiments
    return exp_list

In [35]:
# Implementing our function using the test case defined above:

make_experiment_list(zne_batch_test)

[{'noise_scaling_factors': [1, 1.25, 1.5],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [1, 1.25, 1.5],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'},
 {'noise_scaling_factors': [1, 2, 3],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [1, 2, 3],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'},
 {'noise_scaling_factors': [2, 4, 6],
  'noise_scaling_method': 'global',
  'extrapolation': 'polynomial'},
 {'noise_scaling_factors': [2, 4, 6],
  'noise_scaling_method': 'global',
  'extrapolation': 'linear'}]

## Validating a batch of experiments:


In [36]:
# Validate each experiment in our batch object:

# Create function that takes in a batch dictionary and schema to validate against
def batch_validate(batch_dict, schema):
    # Initialize empty list for validation results
    validation_results = []

    # Create list of experiments
    formatted_batch = make_experiment_list(batch_dict)

    # Iterate over list of experiments and individually validate
    for k in formatted_batch:
        try:
            jsonschema.validate(instance=k, schema=schema)
            result = "validation passed"
        except jsonschema.exceptions.ValidationError as e:
            result = "validation failed"

        # Pull out validation results
        validation_results.append(result)

    return validation_results

In [37]:
# Implementing our batch validate function using the same test case:

batch_validate(zne_batch_test, zne_schema)

['validation passed',
 'validation passed',
 'validation passed',
 'validation passed',
 'validation passed',
 'validation passed']

## Defining the Executor:


In [ ]:
def build_depolarizing_backend(prob=0.005):
    noise_model = NoiseModel()

    depolarizing_err1 = depolarizing_error(prob, num_qubits=1)
    depolarizing_err2 = depolarizing_error(prob, num_qubits=2)
    noise_model.add_all_qubit_quantum_error(depolarizing_err1, ["h", "x", "y", "z"])
    noise_model.add_all_qubit_quantum_error(depolarizing_err2, ["cx"])

    return AerSimulator(noise_model=noise_model)


def make_depolarizing_executor(prob=0.005, shots=4096):
    backend = build_depolarizing_backend(prob)

    def executor(circuit):
        qc = circuit.copy()
        qc.measure_all()
        transpiled_qc = transpile(qc, backend=backend, optimization_level=0)

        result = backend.run(transpiled_qc, shots=shots).result()
        counts = result.get_counts(transpiled_qc)
        total_shots = sum(counts.values())
        zero_state = "0" * qc.num_qubits
        one_state = "1" * qc.num_qubits
        expectation_value = (
            counts.get(zero_state, 0) + counts.get(one_state, 0)
        ) / total_shots

        return expectation_value

    return executor


depolarizing_executor = make_depolarizing_executor(prob=0.01, shots=4096)


## Batch Executor Functions:


In [40]:
# Adjusting Ella's mapping functionality:

noise_scaling_map = {
    "global": fold_global,
    "local_random": fold_gates_at_random,
    "local_all": fold_all,
    "layer": get_layer_folding,
    "identity_scaling": insert_id_layers,
}


def extrapolation_map(single_exp):
    ex_map = {
        "linear": zne.inference.LinearFactory(
            scale_factors=single_exp["noise_scaling_factors"]
        ),
        "richardson": zne.inference.RichardsonFactory(
            scale_factors=single_exp["noise_scaling_factors"]
        ),
        "polynomial": zne.inference.PolyFactory(
            scale_factors=single_exp["noise_scaling_factors"], order=2
        ),
        "exponential": zne.inference.ExpFactory(
            scale_factors=single_exp["noise_scaling_factors"]
        ),
        "poly-exp": zne.inference.PolyExpFactory(
            scale_factors=single_exp["noise_scaling_factors"], order=1
        ),
        "adaptive-exp": zne.inference.AdaExpFactory(
            scale_factor=single_exp["noise_scaling_factors"][1], steps=4, asymptote=None
        ),
    }
    return ex_map[single_exp["extrapolation"]]

In [41]:
# Batch execute function to return list of expectation values:


def batch_execute(batch_dict, circuit, executor):
    # Define list of experiments
    formatted_batch = make_experiment_list(batch_dict)

    # Initialize list to append expectation values into
    exp_val_list = []

    # Iterate over each experiment
    for k in formatted_batch:
        # tmp_executor = functools.partial(executor, noise_model=...)
        exp_val = zne.execute_with_zne(
            circuit=circuit,
            executor=executor,
            factory=extrapolation_map(k),
            scale_noise=noise_scaling_map[k["noise_scaling_method"]],
        )

        # Pull out expectation values
        exp_val_list.append(exp_val)

    return exp_val_list

## Testing:


In [42]:
# Making a 3-qubit GHZ circuit directly in Qiskit:

n = 3
ghz_circ = QuantumCircuit(n)
ghz_circ.h(0)
for idx in range(n - 1):
    ghz_circ.cx(idx, idx + 1)

print(ghz_circ)


     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐     
q_1: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘


In [43]:
ideal_ev = 1.0
noisy_ev = depolarizing_executor(ghz_circ)
print("Ideal EV:", ideal_ev)
print("Depolarizing noisy EV:", noisy_ev)


Ideal EV: 1.0
Depolarizing noisy EV: 0.98974609375


In [44]:
exp_results = batch_execute(zne_batch_test, ghz_circ, depolarizing_executor)

for k in np.arange(1, 7):
    print("Experiment", k, "Mitigated Expectation Value:", exp_results[k - 1])


/Users/nate/code/learn-qem-config/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nate/code/learn-qem-config/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nate/code/learn-qem-config/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nate/code/learn-qem-config/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/nate/code/learn-qem-config/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of no

Experiment 1 Mitigated Expectation Value: 0.8242187499999988
Experiment 2 Mitigated Expectation Value: 1.028645833333334
Experiment 3 Mitigated Expectation Value: 1.0405273437499996
Experiment 4 Mitigated Expectation Value: 0.9956868489583333
Experiment 5 Mitigated Expectation Value: 0.9855957031249994
Experiment 6 Mitigated Expectation Value: 0.9798177083333337
